# local 작업

**import module**

In [95]:
import config
import pickle
import os
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import time
import glob
from pyspark.sql.functions import lit,unix_timestamp
from pyspark import SQLContext

**load .csv file from Local**

In [2]:
# df = sc.textFile("trade_20171214_20171215.csv").persist()
# #tmp = spark.read.csv(
# #     "trade_20171214_20171215.csv", header=True
# # ).persist()

# type(df)
# output[]: pyspark.rdd.RDD

**pyspark.sql.dataframe.DataFrame형태로 load**

In [99]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv("trade_20171214_20171215.csv", header=True)
#df
type(df)

pyspark.sql.dataframe.DataFrame

In [100]:
df.take(1)
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- exchange_rate: string (nullable = true)
 |-- price: string (nullable = true)
 |-- price2: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- total: string (nullable = true)
 |-- type: string (nullable = true)
 |-- exchange: string (nullable = true)
 |-- count: string (nullable = true)
 |-- trade_id: string (nullable = true)



**from MySql**    
    
    [Row(id=1, date=datetime.datetime(2017, 12, 14, 1, 19, 58), exchange_rate=1.0, price=18468000.0, price2=18468000.0, amount=0.0144, 
    total=265939.0, type='buy', exchange='bithumb', count=1, trade_id='ed2dc46f02629d03d98c411a64858b3a37a1b0a42349529c27fe9416')
    
    root
     |-- id: integer (nullable = false)
     |-- date: timestamp (nullable = false)
     |-- exchange_rate: double (nullable = false)
     |-- price: double (nullable = false)
     |-- price2: double (nullable = false)
     |-- amount: double (nullable = false)
     |-- total: double (nullable = false)
     |-- type: string (nullable = false)
     |-- exchange: string (nullable = false)
     |-- count: integer (nullable = false)
     |-- trade_id: string (nullable = true)
    
**from HDFS**

    [Row(id='1', date='2017.12.14 1:19', exchange_rate='1', price='18468000', price2='18468000', amount='0.0144', 
    total='265939', type='buy', exchange='bithumb', count='1', trade_id='ed2dc46f02629d03d98c411a64858b3a37a1b0a42349529c27fe9416')]
    
    root
     |-- id: string (nullable = true)
     |-- date: string (nullable = true)
     |-- exchange_rate: string (nullable = true)
     |-- price: string (nullable = true)
     |-- price2: string (nullable = true)
     |-- amount: string (nullable = true)
     |-- total: string (nullable = true)
     |-- type: string (nullable = true)
     |-- exchange: string (nullable = true)
     |-- count: string (nullable = true)
     |-- trade_id: string (nullable = true)

**Convert the type appropriately**

In [143]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv("trade_20171214_20171215.csv", header=True, nullValue=nullValueList)
#df
type(df)
output_df = df.withColumn("id",df["id"].cast('int'))
output_df = output_df.withColumn('date',unix_timestamp(df['date'],'yyyy-MM-dd HH:mm').cast("timestamp"))
# output_df = output_df.withColumn("date",df["date"].cast('timestamp'))
output_df = output_df.withColumn("exchange_rate",df["exchange_rate"].cast('double'))
output_df = output_df.withColumn("price",df["price"].cast('double'))
output_df = output_df.withColumn("price2",df["price2"].cast('double'))
output_df = output_df.withColumn("amount",df["amount"].cast('double'))
output_df = output_df.withColumn("total",df["total"].cast('double'))
output_df = output_df.withColumn("type",df["type"].cast('string'))
output_df = output_df.withColumn("exchange",df["exchange"].cast('string'))
output_df = output_df.withColumn("count",df["count"].cast('int'))
output_df = output_df.withColumn("trade_id",df["trade_id"].cast('string'))
output_df.printSchema()
# https://stackoverflow.com/questions/45453294/change-the-datatype-of-columns-in-pyspark-dataframe
output_df.take(1)
output_df.show(truncate = True)

root
 |-- id: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- exchange_rate: double (nullable = true)
 |-- price: double (nullable = true)
 |-- price2: double (nullable = true)
 |-- amount: double (nullable = true)
 |-- total: double (nullable = true)
 |-- type: string (nullable = true)
 |-- exchange: string (nullable = true)
 |-- count: integer (nullable = true)
 |-- trade_id: string (nullable = true)

+---+-------------------+-------------+--------+--------+------+---------+----+--------+-----+--------------------+
| id|               date|exchange_rate|   price|  price2|amount|    total|type|exchange|count|            trade_id|
+---+-------------------+-------------+--------+--------+------+---------+----+--------+-----+--------------------+
|  1|0017-12-14 01:19:00|          1.0|1.8468E7|1.8468E7|   0.0| 265939.0| buy| bithumb|    1|ed2dc46f02629d03d...|
|  2|0017-12-14 01:19:00|          1.0|1.8477E7|1.8477E7|   0.0| 822227.0| buy| bithumb|    1|08a225db9a33d

**define class which is Pre-precessing the data**

In [119]:
class Preprocessing_Trading_Data():
    def __init__(self, df, start_time, end_time, time_interval, exchange):
        self.start_time = start_time #'2017-12-14 01:20:00'
        self.end_time = end_time #'2017-12-14 01:20:20'
        self.date_time = start_time
        self.date_time_list = []
        self.date_time_list.append(self.date_time)
        self.time_interval = time_interval
        self.exchange = exchange
        self.df = df
        
    def is_finished(self):
        current_time = datetime.datetime.strptime(self.date_time, '%Y-%m-%d %H:%M:%S')
        end_time = datetime.datetime.strptime(self.end_time, '%Y-%m-%d %H:%M:%S')
        if current_time >= end_time:
            return True
    #- datetime.timedelta(minutes = 1)
    def change_datetime(self, date_time, time_interval):
        date_time = datetime.datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S')
        changed_datetime = date_time + datetime.timedelta(minutes = time_interval)
        self.date_time = changed_datetime.strftime("%Y-%m-%d %H:%M:%S")
        self.date_time_list.append(self.date_time)
        return changed_datetime
    
    def df_divided_by_time_interval(self, date_time, time_interval):
        df_divided = self.df.filter(df.date.between(datetime.datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S'), self.change_datetime(date_time, time_interval)))
        return df_divided
    
    def get_prices_from_df_divided(self, is_fisrt = False):
        df = self.df_divided_by_time_interval(self.date_time, self.time_interval)
        df = df.filter(df.exchange == self.exchange)
        df_open = df.filter(df.id == df.first().id).rdd.map(lambda x: x.price)
        df_max = df.filter(df.id == df.orderBy('price', ascending=False).first().id).rdd.map(lambda x: x.price)
        df_min = df.filter(df.id == df.orderBy('price', ascending=True).first().id).rdd.map(lambda x: x.price)
        df_close = df.filter(df.id == df.orderBy('id', ascending=False).first().id).rdd.map(lambda x: x.price)
        
        df_union = df_open.union(df_max).union(df_min).union(df_close)
        return df_union
    
    def get_date_time_list(self):
        return self.date_time_list

**main function**

In [144]:
dbtable_arg ="TRADE_BTC"
start_time_arg =  '2017-12-14 02:00:00'
end_time_arg =  '2017-12-15 00:00:00'
exchange_arg =  'bithumb'
time_interval_arg = 1
Spark_app_name = dbtable_arg + '|' + start_time_arg + '|' + end_time_arg + '|'  + exchange_arg + '|' + str(time_interval_arg )

In [145]:
if __name__ == '__main__':
    df = output_df
    start_time = start_time_arg
    end_time = end_time_arg
    time_interval = time_interval_arg
    exchange = exchange_arg
    result = []
    data = {}
    data['open'] = []
    data['high'] =  []
    data['low'] =  []
    data['close'] =  []

    
    pickle_file_name = dbtable_arg + '|' + start_time_arg + '|' + end_time_arg + '|'  +exchange_arg + '|' + str(time_interval_arg) + '.pickle'
    
    # file의 존재 유무 확인
    # 중복 수행 방지
    if (os.getcwd() +  '/' +  pickle_file_name in glob.glob(os.getcwd()+"/*.pickle")):
        print("[WARNING] The file Already exist.")
    else:
        trading = Preprocessing_Trading_Data(df, start_time, end_time, time_interval, exchange)
        date_time_list = trading.get_date_time_list()
        while not(trading.is_finished()):
            data_list = trading.get_prices_from_df_divided()
            result.append(list(data_list.collect()))
            
        print(end='\n\n\n\n\n')    
        print("[WARNING] Following result are not same the content of pickle file.")
        print('------------------------------------------------------------------------------')
        print(result)
        date_time_list = date_time_list[:-1]
        print(date_time_list)
        print(len(result))
        print(len(date_time_list))
        print('------------------------------------------------------------------------------')
        print("generating pickle file....",end="\n\n")
        

        data['meta'] = [dbtable_arg, start_time_arg, end_time_arg, exchange_arg, time_interval_arg]
        data['date'] = date_time_list
        for row in result:
            data['open'].append(row[0])
            data['high'].append(row[1])
            data['low'].append(row[2])
            data['close'].append(row[3])
#         data['price'] = result
    
    
        f = open(pickle_file_name, 'wb')
        pickle.dump(data, f)
        f.close()
        print("finished. pickle file is saved at {:s}/".format(os.getcwd()))
        
        
    # f = open(pickle_file_name, 'rb')
    # a = pickle.load(f)
    # print(a)
    print(end='\n\n\n\n\n')   

AttributeError: 'NoneType' object has no attribute 'id'

# debugging

In [150]:
df_open = output_df.filter(output_df.id == output_df.first().id).rdd.map(lambda x: x.price)
df_open.collect()

[18468000.0]

In [153]:
def change_datetime(date_time, time_interval):
        date_time = datetime.datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S')
        changed_datetime = date_time + datetime.timedelta(minutes = time_interval)
        date_time = changed_datetime.strftime("%Y-%m-%d %H:%M:%S")
        date_time_list.append(date_time)
        return changed_datetime

In [151]:
def df_divided_by_time_interval(date_time, time_interval):
    df_divided = output_df.filter(output_df.date.between(datetime.datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S'), \
                                           change_datetime(date_time, time_interval)))
    return df_divided

In [156]:
date_time = '2017-12-14 01:19:58'
time_interval = 1
df_1 = df_divided_by_time_interval(date_time, time_interval)
df_1

DataFrame[id: int, date: timestamp, exchange_rate: double, price: double, price2: double, amount: double, total: double, type: string, exchange: string, count: int, trade_id: string]

In [157]:
df_1.collect()

[]

<img src="https://goo.gl/6EGe5V"></img>

In [158]:
date_time = '2017-12-14 01:19:58'
time_interval = 1
change_datetime(date_time, time_interval)

datetime.datetime(2017, 12, 14, 1, 20, 58)

In [159]:
datetime.datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S')

datetime.datetime(2017, 12, 14, 1, 19, 58)

In [160]:
output_df.date.between(datetime.datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S'), \
                                           change_datetime(date_time, time_interval))

Column<b"((date >= TIMESTAMP('2017-12-14 01:19:58.0')) AND (date <= TIMESTAMP('2017-12-14 01:20:58.0')))">